# Authentication and Initialization
To use NetBrain APIs, you need to authenticate user information to get access token. With the access token, you can specify a tenant/domain to get started with API calls.

In the Authentication category, the following APIs are included:

>* [Log in to get an authentication token](#login)

>* [Log out from a session](#logout)

>* [Get all accessible tenants](#tenants)

>* [Get all accessible domains of a tenant](#domains)

>* Specify a domain to work on

# Log in API
<a herf = '#login'> Use </a> this API call to get an authentication token and start a session. Authentication not required.

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **POST** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/Session  | No |

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|username* | string  | the username to log into your NetBrain domain.  |
|password* | string  | the password to log into your NetBrain domain.  |
|authentication_id | string  | the authentication ID of an external user through SSO, LDAP/AD or TACACS. This parameter is only required for an external user and the value is alias. |

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|token | string | The returned authentication token.  |
|statusCode| integer | The returned status code of executing the API.  |
|statusDescription| string | The explanation of the status code. |

* **Example**


In [20]:
# import python modules 
import requests
import time
import urllib3
import pprint
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set the request parameters
# Eg. User name="NetBrain", Password="Netbrain" for this code sample.
user = "gongdailiu"      
pwd = "1101810209"      
nb_url = "http://192.168.28.79"           

# define a login function 
def loginSession(nb_url, username, password):
    full_url = nb_url + "/ServicesAPI/API/V1/Session"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}    
    try:
        # Do the HTTP request
        response = requests.post(full_url, headers=headers, auth=(username, password), verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            js = response.json()
            return (js)
        else:
            return ("Get token failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
js = loginSession(nb_url, user, pwd)
js# print out the token you got as a string 

{'token': '6bc8df4e-87ce-4291-b9eb-1fb009754092',
 'statusCode': 790200,
 'statusDescription': 'Success.'}

# Log out API
<a herf = '#logout'> Use </a> this API to log out from the current session.

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **DELETE** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/Session  | Yes |

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|token* | string  | input the token received from login.  |

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | The returned status code of executing the API.  |
|statusDescription| string | The explanation of the status code. |

* **Example**


In [19]:
import json
# Set the request parameters
token = "da1e4fcf-b1c6-49cc-849e-fb82d02450da"
nb_url = "http://192.168.28.79"

# Logout NetBrain and release system token.
def logoutSession(nb_url ,token):
    full_url = nb_url + "/ServicesAPI/API/V1/Session"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    data = {"token":token}
    try:
        # Do the HTTP request
        response = requests.delete(full_url, data=json.dumps(data), headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            js = response.json()
            return (js)
        else:
            return ("Session logout failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
result = logoutSession(nb_url ,token)
result# print out the result of whether log out successfully.

{'statusCode': 790200, 'statusDescription': 'Success.'}

# Get all accessible tenants API
<a herf = '#tenants'> Use </a> this function returns a list of accessible tenants (including tenant ID and names). The returned tenant list varies by the user privileges you use to log in. To retrieve a full list of all available tenants, you must log in with admin permissions. Authentication required.

* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **GET** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Tenants | Yes |

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|token* | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |
|nb_URL* | string  | IP address of your NetBrain Web API Server  |

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|tenants | array | A list of all accessible tenants.  |
|tenantId| string | The tenant ID.  |
|tenantName| string | The tenant name. |
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |

* **Example**


In [21]:
# Set the request parameters
token = "6bc8df4e-87ce-4291-b9eb-1fb009754092"
nb_url = "http://192.168.28.79"

# Get all Tenants ID and Name.
def getTenants(nb_url, token):
    full_url = nb_url + "/ServicesAPI/API/V1/CMDB/Tenants"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    try:
        # Do the HTTP request
        response = requests.get(full_url, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return (result)
        else:
            return ("Get tenants failed! - " + str(response.text))
    except Exception as e: return (str(e))

allT = getTenants(nb_url, token)
"""for i in range(len(allT)):
    print(allT[i].tenantName)"""
allT
 # print out the name of all accessible tenants 

{'tenants': [{'tenantId': 'fb24f3f0-81a7-1929-4b8f-99106c23fa5b',
   'tenantName': 'Initial Tenant'}],
 'statusCode': 790200,
 'statusDescription': 'Success.'}

# Get all accessible domains of a tenants API
<a herf = '#domains'> Use </a> this function returns a list of accessible domains in a specific tenant. The returned accessible domains vary by the user privileges you use to log in. To retrieve a full list of domains in a specified tenant, you must log in with system admin or tenant admin permissions.


* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **GET** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/CMDB/Domains | Yes |

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|token* | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |
|tenantId | string  | Unique identifier for the tenant from which you desire to retrieve the domain information. tenantId can be retrieved from get all accessible tenants.  |
|nb_URL* | string  | IP address of your NetBrain Web API Server  |

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|domains | array | A list of all accessible domains. |
|domainId| string | The domain ID.  |
|domainName| string | The domain name. |
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |

* **Example**

In [22]:
# Set the request parameters
token = "6bc8df4e-87ce-4291-b9eb-1fb009754092"
nb_url = "http://192.168.28.79"
tenantId = "fb24f3f0-81a7-1929-4b8f-99106c23fa5b"

# Get all domains info. If tenantId is None, will present all domains.
def getDomains(nb_url, token, tenantId=None):
    full_url = nb_url + "/ServicesAPI/API/V1/CMDB/Domains"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    data = {"tenantId": tenantId}
    try:
        # Do the HTTP request
        response = requests.get(full_url, params=data, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return (result)
        else:
            return ("Get domains failed! - " + str(response.text))
    except Exception as e: return (str(e))
    
allD = getDomains(nb_url, token)
"""for i in range(len(allD)):
    print(allD[i].domainName)"""
allD # print out all accessible tenants in json

{'domains': [{'domainId': '850ff5e9-c639-404d-85a3-d920dbee509c',
   'domainName': 'Support and Service'},
  {'domainId': '0201adc4-ae96-46f0-ae3d-01cdba9e41d6',
   'domainName': 'GE Test'}],
 'statusCode': 790200,
 'statusDescription': 'Success.'}

# Specify a domain to work on API
<a herf = '#domains'> Use </a> this API to specify a domain to work on to get or set NetBrain data by associating domainID to the current session.


* **Resource Information**

|**Method**|**URL**|**Authentication**|
|------|------|------|
| **PUT** | http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API/V1/Session/CurrentDomain | Yes |

* **Header**

|**Parameter**|**Type**|**Description**|
|------|------|------|
| Content-Type | string  | support "application/json" |
| Accept | string  | support "application/json" |

* **Parameters**(*required)

|**Name**|**Type**|**Description**|
|------|------|------|
|token* | string  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |
|tenantId* | string  | Unique identifier for the tenant from which you desire to retrieve the domain information. tenantId can be retrieved from get all accessible tenants.  |
|domainId* | string  | input the ID of the target domain. Get a domain ID by using the API [Get all accessible domains of a tenant.](https://www.netbraintech.com/docs/ie71/help/get-all-accessible-domains-of-tenant.htm)   |
|nb_URL* | string  | IP address of your NetBrain Web API Server  |

* **Response Information**

|**Name**|**Type**|**Description**|
|------|------|------|
|statusCode| integer | Code issued by NetBrain server indicating the execution result.  |
|statusDescription| string | The explanation of the status code. |

* **Example**

In [23]:
# Set the request parameters
token = "6bc8df4e-87ce-4291-b9eb-1fb009754092"
nb_url = "http://192.168.28.79"
tenantId = "fb24f3f0-81a7-1929-4b8f-99106c23fa5b"
domainId = "0201adc4-ae96-46f0-ae3d-01cdba9e41d6"

# Login selected domain
def loginDomain(nb_url, token, tenantId, domainId):
    full_url = nb_url + "/ServicesAPI/API/V1/Session/CurrentDomain"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    body = {
            "tenantId": tenantId,
            "domainId": domainId
        }
    try:
        # Do the HTTP request
        response = requests.put(full_url, data=json.dumps(body), headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return (result)
        elif response.status_code != 200:
            return ("Login failed! - " + str(response.text))
    except Exception as e: return (str(e))

result = loginDomain(nb_url, token, tenantId, domainId)
print(result)# print out the login result of one particular domain. 

{'statusCode': 790200, 'statusDescription': 'Success.'}
